In [1]:
import pickle 
import os
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
model_path = r"C:\seoul_bike_sharing_demand_prediction\models\xgboost_regressor_r2_0_946_v1.pkl"

model = pickle.load(open(model_path, "rb"))

In [3]:
data = [
       [ 1.07807977,  0.22192612,  0.77468297,  1.82005277, -1.26200749,
       -0.25194272, -0.12916836, -0.17535463, -0.22754885, -5.47803337,
       -0.64744078, -0.43640293,  0.30459857,  1.74132735, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 ,  2.46842407,
       -0.40081071, -0.41367015],
       [-0.65432992,  1.24504573,  0.23697162, -0.5001855 ,  0.56154562,
       -0.28664608, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        0.94701966,  0.14260346,  0.30459857, -0.57427456,  1.71961809,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
        2.49494334, -0.41367015],
       [ 0.50060987,  0.98718632, -0.69180254,  2.40011233,  0.92262892,
        1.56419996, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
       -0.98911087,  0.72160984,  0.30459857, -0.57427456, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
       -0.40081071, -0.41367015]
       ]

prediction = model.predict(data)
list(prediction)

[-48.7018, 1263.8368, 1451.4027]

## Load SC 

In [4]:
sc_dump_path = r"C:\seoul_bike_sharing_demand_prediction\models\sc.pkl"

sc = pickle.load(open(sc_dump_path, "rb"))

In [5]:
sc

StandardScaler()

## Users Input

In [61]:
date = "11/06/2023"
hour = 14
temperature = 25
humidity = 91
wind_speed = 50
visibility = 70
solar_radiation = 3
rainfall = 30
snowfall = 0.0
seasons = "Spring"
holiday = "Holiday"
functioning_day = "No"

## Converting User Data into Model Consumable Formate

In [62]:
holiday_dic = {"No Holiday": 0, "Holiday": 1}
functioning_dic = {"No":0, "Yes":1}

def get_string_to_datetime(date):
    dt = datetime.strptime(date, "%d/%m/%Y")
    return {"day": dt.day, "month": dt.month, "year": dt.year, "week_day": dt.strftime("%A")}

str_to_date = get_string_to_datetime(date)
str_to_date

{'day': 11, 'month': 6, 'year': 2023, 'week_day': 'Sunday'}

In [63]:
u_input_list = [hour, temperature, humidity, wind_speed, visibility, solar_radiation, rainfall, snowfall,
               holiday_dic[holiday], functioning_dic[functioning_day],
                str_to_date["day"], str_to_date["month"], str_to_date["year"]]

features_name = ["Hour", 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day', 'Month', 'Year']

df_u_input = pd.DataFrame([u_input_list], columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,14,25,91,50,70,3,30,0.0,1,0,11,6,2023


In [64]:
def season_to_df(seasons):
     seasons_cols = ['Spring', 'Summer', 'Winter']
     seasons_data = np.zeros((1, len(seasons_cols)), dtype="int")

     df_seasons = pd.DataFrame(seasons_data, columns=seasons_cols)
     if seasons in seasons_cols:
         df_seasons[seasons]=1     
     return df_seasons

df_seasons = season_to_df(seasons)
df_seasons

,Spring,Summer,Winter
0,1,0,0


In [65]:
def days_df(week_day):
     days_names = ['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
     days_name_data = np.zeros((1, len(days_names)), dtype="int")

     df_days = pd.DataFrame(days_name_data, columns=days_names)
     if week_day in days_names:
         df_days[week_day]=1     
     return df_days

df_days = days_df(str_to_date["week_day"])
df_days

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,1,0,0,0


In [66]:
df_for_pred = pd.concat([df_u_input, df_seasons, df_days], axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,14,25,91,50,70,3,30,0.0,1,0,...,2023,1,0,0,0,0,1,0,0,0


In [67]:
sc_data_for_pred = sc.transform(df_for_pred)
sc_data_for_pred

array([[ 0.3562424 ,  1.01214045,  1.60569143, 46.67799252, -2.25952072,
         2.81352104, 25.27434606, -0.17535463,  4.39466079, -5.47803337,
        -0.53355074, -0.14689974, 18.24263884,  1.74132735, -0.58152447,
        -0.57866842, -0.40630934, -0.40916536,  2.45398535, -0.40511677,
        -0.40081071, -0.41367015]])

## Prediction in deployment

In [68]:
model.predict(sc_data_for_pred)

print(f"Rented Bike Demand On date : {date}, and Time: {hour} is : {round(model.predict(sc_data_for_pred).tolist()[0])}")

Rented Bike Demand On date : 11/06/2023, and Time: 14 is : 303
